# Αλγόριθμος ΚΝΝ

Στο παρακάτω παράδειγμα θα εξετάσουμε πως λειτουργεί ο αλγόριθμος Κ-Nearest Neighbors ([KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm?msclkid=0c75f966cf9c11ecbab5cba311a90428)), χρησιμοποιώντας το [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set?msclkid=33057869cf9c11ec8e488a45734cbc68) σε προβλήματα κατηγοριοποίησης και προβλέψεων. Το αρχείο που θα χρησιμοποιήσετε είναι διαθέσιμο [**εδώ**](https://github.com/nkostopoulos/StochasticsLabPublic/blob/master/lab9/KNN/iris.csv).


### Ερωτήσεις



*   Να περιγράψετε συνοπτικά το τρόπο λειτουργίας του αλγορίθμου ΚΝΝ
*   Ποια είναι τα πλεονεκτήματα και τα μειονεκτήματα του συγκεκριμένου αλγορίθμου;
*   Ο συγκεκριμένος αλγόριθμος είναι αποδοτικός στην περίπτωση που έχουμε μεγάλο πλήθος χαρακτηριστικών; Τι συμβαίνει στις περιπτώσεις όπου επιλέξουμε μεγάλο ή μικρό k;



In [1]:
# Make Predictions with k-nearest neighbors on the Iris Flowers Dataset
from csv import reader
from math import sqrt

In [2]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])



*   Αναφέρετε άλλες μεθόδους υπολογισμού της απόστασης



ΑΠΑΝΤΗΣΗ:

1.Ευκλείδεια απόσταση: η καρτεσιανή απόσταση μεταξύ των δύο σημείων που βρίσκονται στο επίπεδο/υπερεπίπεδο. Η Ευκλείδεια απόσταση μπορεί επίσης να απεικονιστεί ως το μήκος της ευθείας που ενώνει τα δύο σημεία που λαμβάνονται υπόψη. Αυτή η μέτρηση μας βοηθά να υπολογίσουμε την καθαρή μετατόπιση που γίνεται μεταξύ των δύο καταστάσεων ενός αντικειμένου.

2.Απόσταση Μανχάταν: η μέτρηση της απόστασης του Μανχάταν χρησιμοποιείται γενικά όταν μας ενδιαφέρει η συνολική απόσταση που διανύει το αντικείμενο αντί για τη μετατόπιση. Αυτή η μέτρηση υπολογίζεται αθροίζοντας την απόλυτη διαφορά μεταξύ των συντεταγμένων των σημείων σε n-διαστάσεις.

3.Απόσταση Minkowski: Μπορούμε να πούμε ότι η Ευκλείδεια, όπως και η απόσταση του Μανχάταν, είναι ειδικές περιπτώσεις της απόστασης Minkowski .

In [8]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction



*   Δοκιμάστε με τα παρακάτω ζεύγη τιμών και σημειώστε τα αποτελέσματα


> *   [4.9, 3.1, 1.5, 0.1]
*   [6.9, 3.1, 4.9, 1.5]
*   [5.0, 2.0, 3.5, 1.0]
*   [5.6, 2.7, 4.2, 1.3]
*   [6.3, 3.3, 6.0, 2.5]
*   [5.7, 2.9, 4.2, 1.3]
*   [5.9, 3.0, 5.1, 1.8]











In [9]:
# Make a prediction with KNN on Iris Dataset
filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# define model parameter
num_neighbors = 5
# define a new record
row = [4.5, 2.3, 1.3, 0.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

[Iris-versicolor] => 0
[Iris-virginica] => 1
[Iris-setosa] => 2
Data=[4.5, 2.3, 1.3, 0.3], Predicted: 2


In [10]:
test_rows = [[4.9, 3.1, 1.5, 0.1],
             [6.9, 3.1, 4.9, 1.5],
             [5.0, 2.0, 3.5, 1.0],
             [5.6, 2.7, 4.2, 1.3],
             [6.3, 3.3, 6.0, 2.5],
             [5.7, 2.9, 4.2, 1.3],
             [5.9, 3.0, 5.1, 1.8]]

for row in test_rows:
    label = predict_classification(dataset, row, num_neighbors)
    print('Data=%s, Predicted: %s' % (row, label))

Data=[4.9, 3.1, 1.5, 0.1], Predicted: 2
Data=[6.9, 3.1, 4.9, 1.5], Predicted: 0
Data=[5.0, 2.0, 3.5, 1.0], Predicted: 0
Data=[5.6, 2.7, 4.2, 1.3], Predicted: 0
Data=[6.3, 3.3, 6.0, 2.5], Predicted: 1
Data=[5.7, 2.9, 4.2, 1.3], Predicted: 0
Data=[5.9, 3.0, 5.1, 1.8], Predicted: 1




*   Δοκιμάστε να υπολογίσετε την απόσταση με τη μετρική [Manhattan](https://iq.opengenus.org/manhattan-distance/#:~:text=Manhattan%20distance%20is%20a%20distance%20metric%20between%20two,the%20measures%20in%20all%20dimensions%20of%20two%20points.?msclkid=50bbf70ecfa011ec91862d6b9263d761) για τα ζέυγη που σας έχουν δοθεί. Παρατηρείτε κάποια διαφοροποίηση ως προς τα αποτελέσματα;



In [16]:
def manhattan_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):  # Υποθέτουμε ότι η τελευταία στήλη είναι η ετικέτα/κατηγορία
        distance += abs(row1[i] - row2[i])
    return distance

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = manhattan_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

In [17]:
# Make a prediction with KNN on Iris Dataset
filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# define model parameter
num_neighbors = 5
# define a new record
row = [4.5, 2.3, 1.3, 0.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

[Iris-versicolor] => 0
[Iris-virginica] => 1
[Iris-setosa] => 2
Data=[4.5, 2.3, 1.3, 0.3], Predicted: 2


In [18]:
test_rows = [[4.9, 3.1, 1.5, 0.1],
             [6.9, 3.1, 4.9, 1.5],
             [5.0, 2.0, 3.5, 1.0],
             [5.6, 2.7, 4.2, 1.3],
             [6.3, 3.3, 6.0, 2.5],
             [5.7, 2.9, 4.2, 1.3],
             [5.9, 3.0, 5.1, 1.8]]

for row in test_rows:
    label = predict_classification(dataset, row, num_neighbors)
    print('Data=%s, Predicted: %s' % (row, label))

Data=[4.9, 3.1, 1.5, 0.1], Predicted: 2
Data=[6.9, 3.1, 4.9, 1.5], Predicted: 0
Data=[5.0, 2.0, 3.5, 1.0], Predicted: 0
Data=[5.6, 2.7, 4.2, 1.3], Predicted: 0
Data=[6.3, 3.3, 6.0, 2.5], Predicted: 1
Data=[5.7, 2.9, 4.2, 1.3], Predicted: 0
Data=[5.9, 3.0, 5.1, 1.8], Predicted: 1


ΣΧΟΛΙΟ:

Με βάση τα παραπάνω αποτελέσματα, δεν υπάρχει κάποια αλλαγή, δηλαδή οι 2 αλγόριθμοι δώσανε το ίδιο αποτέλεσμα.

ΑΠΑΝΤΗΣΕΙΣ ΘΕΩΡΙΑΣ:

i)

Ο αλγόριθμος (K-NN) είναι ένας ευέλικτος και ευρέως χρησιμοποιούμενος αλγόριθμος μηχανικής μάθησης που χρησιμοποιείται κυρίως για την απλότητα και την ευκολία εφαρμογής του. Δεν απαιτεί υποθέσεις σχετικά με την υποκείμενη διανομή δεδομένων. Μπορεί επίσης να χειριστεί τόσο αριθμητικά όσο και κατηγορικά δεδομένα, καθιστώντας το μια ευέλικτη επιλογή για διάφορους τύπους συνόλων δεδομένων σε εργασίες ταξινόμησης και παλινδρόμησης. Είναι μια μη παραμετρική μέθοδος που κάνει προβλέψεις με βάση την ομοιότητα των σημείων δεδομένων σε ένα δεδομένο σύνολο δεδομένων. Το K-NN είναι λιγότερο ευαίσθητο σε ακραίες τιμές σε σύγκριση με άλλους αλγόριθμους.

Ο αλγόριθμος K-NN λειτουργεί βρίσκοντας τους K πλησιέστερους γείτονες σε ένα δεδομένο σημείο δεδομένων με βάση μια μέτρηση απόστασης, όπως η Ευκλείδεια απόσταση. Η κλάση ή η τιμή του σημείου δεδομένων καθορίζεται στη συνέχεια από την πλειοψηφία ή τον μέσο όρο των γειτόνων Κ. Αυτή η προσέγγιση επιτρέπει στον αλγόριθμο να προσαρμόζεται σε διαφορετικά μοτίβα και να κάνει προβλέψεις με βάση την τοπική δομή των δεδομένων.

Για να προσδιορίσουμε τις πλησιέστερες ομάδες ή τα πλησιέστερα σημεία για ένα σημείο στον αλγόριθμο αυτό χρειαζόμαστε κάποια μέτρηση. Για το σκοπό αυτό, χρησιμοποιούμε τις παρακάτω μετρήσεις απόστασης:


1. Ευκλείδεια απόσταση: η καρτεσιανή απόσταση μεταξύ των δύο σημείων που βρίσκονται στο επίπεδο/υπερεπίπεδο. Η Ευκλείδεια απόσταση μπορεί επίσης να απεικονιστεί ως το μήκος της ευθείας που ενώνει τα δύο σημεία που λαμβάνονται υπόψη. Αυτή η μέτρηση μας βοηθά να υπολογίσουμε την καθαρή μετατόπιση που γίνεται μεταξύ των δύο καταστάσεων ενός αντικειμένου.

2. Απόσταση Μανχάταν: η μέτρηση της απόστασης του Μανχάταν χρησιμοποιείται γενικά όταν μας ενδιαφέρει η συνολική απόσταση που διανύει το αντικείμενο αντί για τη μετατόπιση. Αυτή η μέτρηση υπολογίζεται αθροίζοντας την απόλυτη διαφορά μεταξύ των συντεταγμένων των σημείων σε n-διαστάσεις.

3. Απόσταση Minkowski: Μπορούμε να πούμε ότι η Ευκλείδεια, όπως και η απόσταση του Μανχάταν, είναι ειδικές περιπτώσεις της απόστασης Minkowski .

Συνοπτικά: για να εφαρμόσουμε τον αλγόριθμο K-NN πρέπει να εφαρμόσουμε τα παρακάτω βήματα:

1.  Επιλογή της βέλτιστης τιμής του K: Το K αντιπροσωπεύει τον αριθμό των πλησιέστερων γειτόνων που πρέπει να ληφθούν υπόψη κατά την πρόβλεψη.
2. Υπολογισμός απόστασης: Για τη μέτρηση της ομοιότητας μεταξύ σημείων δεδομένων στόχου και εκπαίδευσης, χρησιμοποιείται η Ευκλείδεια απόσταση. Η απόσταση υπολογίζεται μεταξύ καθενός από τα σημεία δεδομένων στο σύνολο δεδομένων και το σημείο στόχου.
3.Εύρεση πλησιέστερων γειτόνων: Τα k σημεία δεδομένων με τις μικρότερες αποστάσεις από το σημείο στόχο είναι οι πλησιέστεροι γείτονες
4. Απόφαση για ταξινόμηση ή λήψη μέσου όρου για παλινδρόμηση:
Στο πρόβλημα ταξινόμησης, οι ετικέτες κλάσης του καθορίζονται με πλειοψηφία. Η κλάση με τις περισσότερες εμφανίσεις μεταξύ των γειτόνων γίνεται η προβλεπόμενη κλάση για το σημείο δεδομένων στόχου.
Στο πρόβλημα παλινδρόμησης, η ετικέτα κλάσης υπολογίζεται λαμβάνοντας τον μέσο όρο των τιμών στόχου των K πλησιέστερων γειτόνων. Η υπολογισμένη μέση τιμή γίνεται η προβλεπόμενη έξοδος για το σημείο δεδομένων στόχου.

ii)

Πλεονεκτήματα του αλγορίθμου KNN

1. Εύκολο στην εφαρμογή καθώς η πολυπλοκότητα του αλγορίθμου δεν είναι τόσο υψηλή.
2. Προσαρμόζεται εύκολα --> Σύμφωνα με τη λειτουργία του αλγόριθμου KNN, αποθηκεύει όλα τα δεδομένα στην αποθήκευση μνήμης και επομένως κάθε φορά που προστίθεται ένα νέο παράδειγμα ή σημείο δεδομένων, ο αλγόριθμος προσαρμόζεται σύμφωνα με αυτό το νέο παράδειγμα και έχει τη συμβολή του και στις μελλοντικές προβλέψεις .
3. Λίγες υπερπαράμετροι – Οι μόνες παράμετροι που απαιτούνται στην εκπαίδευση ενός αλγόριθμου KNN είναι η τιμή του k και η επιλογή της μέτρησης απόστασης που θα θέλαμε να επιλέξουμε από τη μέτρηση αξιολόγησής μας.

Μειονεκτήματα του αλγορίθμου KNN

1. Δεν κλιμακώνεται --> Όπως έχουμε ακούσει για αυτό ότι ο αλγόριθμος KNN θεωρείται επίσης ένας αλγόριθμος Lazy. Η κύρια σημασία αυτού του όρου είναι ότι απαιτεί μεγάλη υπολογιστική ισχύ καθώς και αποθήκευση δεδομένων. Αυτό καθιστά αυτόν τον αλγόριθμο τόσο χρονοβόρο όσο και εξαντλητικό των πόρων.
2. Curse of Dimensionality – Υπάρχει ένας όρος γνωστός ως φαινόμενο κορύφωσης σύμφωνα με αυτό, ο αλγόριθμος KNN επηρεάζεται από την κατάρα της διάστασης που υποδηλώνει ότι ο αλγόριθμος αντιμετωπίζει έναν δύσκολο χρόνο να ταξινομήσει σωστά τα σημεία δεδομένων όταν η διάσταση είναι πολύ υψηλή.
3. Επιρρεπής σε υπερπροσαρμογή – Καθώς ο αλγόριθμος επηρεάζεται λόγω της κατάρας της διαστασιοποίησης, είναι επίσης επιρρεπής στο πρόβλημα της υπερπροσαρμογής. Ως εκ τούτου, γενικά η επιλογή χαρακτηριστικών καθώς και οι τεχνικές μείωσης διαστάσεων εφαρμόζονται για την αντιμετώπιση αυτού του προβλήματος.

iii)

Ο αλγόριθμος k-Nearest Neighbors (KNN) μπορεί να παρουσιάσει σημαντικά ζητήματα απόδοσης όταν εφαρμόζεται σε σύνολα δεδομένων με μεγάλο πλήθος χαρακτηριστικών, φαινόμενο γνωστό ως «curse of dimensionality» (κατάρα της διαστατικότητας).

Curse of Dimensionality: Όταν ο αριθμός των χαρακτηριστικών (διαστάσεων) είναι πολύ μεγάλος, η απόσταση μεταξύ των σημείων δεδομένων μπορεί να γίνεται λιγότερο διακριτή. Οι περισσότερες μετρικές απόστασης, όπως η Ευκλείδια απόσταση, γίνονται λιγότερο αποτελεσματικές, καθώς οι αποστάσεις τείνουν να συγκλίνουν σε παρόμοιες τιμές. Αυτό καθιστά δύσκολη την αναγνώριση των πλησιέστερων γειτόνων.

Ο KNN απαιτεί τον υπολογισμό της απόστασης μεταξύ του σημείου προς ταξινόμηση και κάθε σημείου στο σύνολο εκπαίδευσης. Όταν τα χαρακτηριστικά είναι πολλά, αυτός ο υπολογισμός γίνεται πιο απαιτητικός σε πόρους και χρόνο.

Το πρόβλημα αυτό μπορεί να αντιμετωπιστεί με την εφαρμογή τεχνικών μείωσης των διαστάσεων, όπως η Ανάλυση Κύριων Συνιστωσών (PCA) ή η επιλογή χαρακτηριστικών, για να βελτιωθεί η απόδοση του KNN σε σύνολα δεδομένων με μεγάλο πλήθος χαρακτηριστικών.


Μικρή τιμή του k(k=1 ή 2):

Πλεονεκτήματα: Ο αλγόριθμος μπορεί να είναι πιο ευαίσθητος και να ανιχνεύει καλύτερα τις τοπικές δομές στα δεδομένα. Μπορεί να επιτύχει υψηλή ακρίβεια σε καλά διαχωρισμένα σύνολα δεδομένων.

Μειονεκτήματα: Αυξάνεται η ευαισθησία στον θόρυβο και τις ανωμαλίες(δλαδή πληροφορίες ή σημεία δεδομένων που είναι άσχετα ή τυχαία και δεν ακολουθούν τα βασικά πρότυπα ή τάσεις που παρουσιάζει το υπόλοιπο σύνολο δεδομένων). Τα αποτελέσματα μπορεί να είναι ασταθή, καθώς ένα μεμονωμένο σημείο θορύβου μπορεί να επηρεάσει σημαντικά την απόφαση ταξινόμησης.


Μεγάλη τιμή του k (k = 15 ή 20):

Πλεονεκτήματα: Ο αλγόριθμος γίνεται πιο σταθερός και λιγότερο επηρεασμένος από τον θόρυβο. Οι αποφάσεις ταξινόμησης βασίζονται σε μεγαλύτερο αριθμό γειτόνων, γεγονός που μπορεί να βελτιώσει τη γενίκευση.

Μειονεκτήματα: Η αυξημένη τιμή του k μπορεί να προκαλέσει υπερβολική εξομάλυνση των αποφάσεων ταξινόμησης, μειώνοντας την ικανότητα του αλγορίθμου να αναγνωρίζει λεπτές τοπικές δομές και διαφοροποιήσεις στα δεδομένα. Αυτό μπορεί να οδηγήσει σε χαμηλότερη ακρίβεια σε σύνολα δεδομένων με πολύπλοκα πρότυπα.